In [23]:
import json
import os
from statistics import mode, multimode
from typing import Dict, List, Tuple, Union

import numpy as np
import torch
from torch.nn import functional as F
from torchvision import transforms as T

In [102]:
gallery_set = torch.randn(size=(100, 128))
gallery_label = torch.randint(low=0, high=10, size=(100, ))
query_set = torch.randn(size=(10, 128))
query_label = torch.randint(low=0, high=10, size=(10, ))

In [109]:
def get_top_k(query: torch.Tensor, gallery: torch.Tensor, top_k: int = 1):
    similarity = F.cosine_similarity(x1=query, x2=gallery, dim=-1)
    ranking = torch.argsort(input=similarity,
                            stable=True,
                            descending=True,
                            dim=-1)
    return ranking[:top_k]


def cummulative_matching_characteristics(query_set: torch.Tensor,
                                         query_label: torch.Tensor,
                                         gallery_set: torch.Tensor,
                                         gallery_label: torch.Tensor,
                                         top_k: int = 10):
    similarity_socores = torch.stack(list(
        map(
            lambda s: get_top_k(
                query=s.unsqueeze(0), gallery=gallery_set, top_k=top_k),
            query_set)),
                                     dim=0)
    nearest_labels = torch.stack(list(
        map(lambda s: gallery_label[s], similarity_socores)),
                                 dim=0)
    labels = torch.mode(input=nearest_labels, dim=-1).values
    cmc_values = torch.where(labels == query_label, 1, 0)
    cmc_masks = torch.where(labels == query_label, 1, 0)
    cmc_values = torch.sum(cmc_masks) / len(cmc_masks)
    return cmc_values

In [2]:
LTCC_PATH = "/Users/jurgendn/Downloads/LTCC_ReID/"

In [8]:
QUERY_PATH =  os.path.join(LTCC_PATH, 'query')

In [12]:
QUERY_SET = "/Users/jurgendn/Downloads/ltcc_metadata/query.json"
GALLERY_SET = "/Users/jurgendn/Downloads/ltcc_metadata/gallery.json"

In [15]:
with open(QUERY_SET, "r") as f:
    query_set = json.load(f)
    
with open(GALLERY_SET, "r") as f:
    gallery_set = json.load(f)

In [16]:
gallery_set

[{'img_path': 'test/074_5_c6_009566.png',
  'p_id': 74,
  'cam_id': 6,
  'cloth_id': 5,
  'pose_landmarks': [[0.5626741647720337,
    0.14451566338539124,
    -0.12371525913476944,
    0.9999947547912598],
   [0.4066626727581024,
    0.21948449313640594,
    0.15468548238277435,
    0.999853253364563],
   [0.4373708963394165,
    0.22203704714775085,
    -0.39668986201286316,
    0.9999879598617554],
   [0.5742307305335999,
    0.3458893299102783,
    -0.612220823764801,
    0.9931458234786987],
   [0.8473266363143921,
    0.4400610625743866,
    -0.5327064394950867,
    0.9866821765899658],
   [0.37595444917678833,
    0.21693193912506104,
    0.7060608267784119,
    0.9997184872627258],
   [0.39769911766052246,
    0.33270081877708435,
    1.0673632621765137,
    0.013678119517862797],
   [0.5907445549964905,
    0.4037688970565796,
    1.135157585144043,
    0.054349325597286224],
   [0.35419419407844543,
    0.49596357345581055,
    -0.3381711542606354,
    0.9999492168426514],
   

In [24]:
def compute_mAP(index: Union[np.ndarray, torch.Tensor],
                good_index: Union[np.ndarray, torch.Tensor],
                junk_index: Union[np.ndarray, torch.Tensor]):
    """
    compute_mAP _summary_

    Parameters
    ----------
    index : Union[np.ndarray, torch.Tensor]
        1d array
    good_index : Union[np.ndarray, torch.Tensor]
        1d array
    junk_index : Union[np.ndarray, torch.Tensor]
        1d array

    Returns
    -------
    _type_
        1d array
    """    
    ap = 0
    cmc = torch.IntTensor(len(index)).zero_()
    if good_index.size == 0:  # if empty
        cmc[0] = -1
        return ap, cmc

    # remove junk_index
    mask = np.in1d(index, junk_index, invert=True)
    index = index[mask]

    # find good_index index
    ngood = len(good_index)
    mask = np.in1d(index, good_index)
    rows_good = np.argwhere(mask == True)
    rows_good = rows_good.flatten()

    cmc[rows_good[0]:] = 1
    for i in range(ngood):
        d_recall = 1.0 / ngood
        precision = (i + 1) * 1.0 / (rows_good[i] + 1)
        if rows_good[i] != 0:
            old_precision = i * 1.0 / rows_good[i]
        else:
            old_precision = 1.0
        ap = ap + d_recall * (old_precision + precision) / 2

    return ap, cmc

In [26]:
def evaluate(query_features: torch.Tensor, query_labels: torch.Tensor,
             query_clothes: torch.Tensor, gallery_features: torch.Tensor,
             gallery_labels: torch.Tensor, gallery_clothes: torch.Tensor):
    query = query_features.view(-1, 1)
    # print(query.shape)
    score = torch.mm(gallery_features, query)
    # Cosine similarity/distance
    score = score.squeeze(1).cpu()
    score = score.numpy()
    # predict index
    index = np.argsort(score)
    # Ranking, by distance
    index = index[::-1]
    # index = index[0:2000]
    # good index
    query_index = np.argwhere(gallery_labels == query_labels)
    camera_index = np.argwhere(gallery_clothes == query_clothes)

    # Same person, different camera, with label
    good_index = np.setdiff1d(query_index, camera_index, assume_unique=True)
    # No label
    junk_index1 = np.argwhere(gallery_labels == -1)
    # Same person, same camera id
    junk_index2 = np.intersect1d(query_index, camera_index)
    junk_index = np.append(junk_index2, junk_index1)  #.flatten())

    CMC_tmp = compute_mAP(index, good_index, junk_index)
    return CMC_tmp